In [64]:
import numpy as np
import pandas as pd

from litedl.utils import OneHotEncoder, MinMaxScaler, data_split, get_batches, Standardizer
from litedl.models import FeedForwardNeuralNetwork
from litedl.layers import Affine, Sigmoid, SoftmaxWithLoss, ReLU
from litedl.optimizers import SGD

epoch = 100
batch_size = 100

dataset = pd.read_csv('dataset/heart_attack_risk_dataset.csv')
feature_dataset = dataset.iloc[:, :-1]
label_dataset = dataset.iloc[:, -1]

categorical = ['Gender', 'Physical_Activity_Level', 'Stress_Level', 'Chest_Pain_Type', 'Thalassemia', 'ECG_Results']
binary = ['Smoking', 'Alcohol_Consumption', 'Diabetes', 'Hypertension', 'Family_History', 'Fasting_Blood_Sugar', 'Exercise_Induced_Angina']
num = ['Age', 'BMI', 'Cholesterol_Level', 'Resting_BP', 'Heart_Rate', 'Max_Heart_Rate_Achieved']

categorical_dataset = pd.get_dummies(feature_dataset[categorical]).astype(np.int32)
categorical_data = categorical_dataset.values

binary_data = feature_dataset[binary].values
binary_dataset = pd.DataFrame(binary_data, columns=binary)

num_data = feature_dataset[num].values
num_scaler = MinMaxScaler()
num_scaler.fit(num_data)
num_data = num_scaler.transform(num_data)
num_dataset = pd.DataFrame(num_data, columns=num)

label_data = label_dataset.values.reshape(-1, 1)
label_encoder = OneHotEncoder()
label_encoder.fit(label_data)
labels = label_encoder.transform(label_data)

feature_dataset = pd.concat([categorical_dataset, binary_dataset, num_dataset], axis=1)
features = feature_dataset.values

train_features, test_features = data_split(features, 0.2)
train_labels, test_labels = data_split(labels, 0.2)

feature_batches = get_batches(train_features, batch_size)
label_batches = get_batches(train_labels, batch_size)

In [65]:
model = FeedForwardNeuralNetwork()
affine1 = Affine(input_size=31, output_size=3)
relu1 = ReLU()
affine2 = Affine(input_size=100, output_size=50)
relu2 = ReLU()
affine3 = Affine(input_size=50, output_size=3)
softmax_loss = SoftmaxWithLoss()
sgd = SGD()

model.add_layer(affine1)
'''model.add_layer(relu1)
model.add_layer(affine2)
model.add_layer(relu2)
model.add_layer(affine3)'''
model.add_loss_layer(softmax_loss)

for i in range(epoch):
    losses = []
    for j in range(len(feature_batches)):
        losses.append(model.forward(feature_batches[j], label_batches[j]))
        model.backward()
        model.step(optimizer=sgd)
        
    print(f'epoch {i}: {sum(losses) / len(losses)}')

epoch 0: -9.99999950583871e-08
epoch 1: -9.99999950583871e-08
epoch 2: -9.99999950583871e-08
epoch 3: -9.99999950583871e-08
epoch 4: -9.99999950583871e-08
epoch 5: -9.99999950583871e-08
epoch 6: -9.99999950583871e-08
epoch 7: -9.99999950583871e-08
epoch 8: -9.99999950583871e-08
epoch 9: -9.99999950583871e-08
epoch 10: -9.99999950583871e-08
epoch 11: -9.99999950583871e-08
epoch 12: -9.99999950583871e-08
epoch 13: -9.99999950583871e-08
epoch 14: -9.99999950583871e-08
epoch 15: -9.99999950583871e-08
epoch 16: -9.99999950583871e-08
epoch 17: -9.99999950583871e-08
epoch 18: -9.99999950583871e-08
epoch 19: -9.99999950583871e-08
epoch 20: -9.99999950583871e-08
epoch 21: -9.99999950583871e-08
epoch 22: -9.99999950583871e-08
epoch 23: -9.99999950583871e-08
epoch 24: -9.99999950583871e-08
epoch 25: -9.99999950583871e-08
epoch 26: -9.99999950583871e-08
epoch 27: -9.99999950583871e-08
epoch 28: -9.99999950583871e-08
epoch 29: -9.99999950583871e-08
epoch 30: -9.99999950583871e-08
epoch 31: -9.99999

In [66]:
test_prediction = model.predict(test_features)
test_prediction = np.argmax(test_prediction, axis=1)
test_answer = np.argmax(test_labels, axis=1)

same_count = np.sum(test_prediction == test_answer)
accuracy = same_count / test_prediction.shape[0]

print(f'정확도: {accuracy}')

정확도: 0.5024
